In [1]:
import os

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from IPython import display
from datetime import datetime
from IPython.display import clear_output

import MeaslesDataLoader as md
import MeaslesModelEval as mm
import EpiPreprocessor as ep

from xgboost import XGBRegressor


%matplotlib inline

/Users/ameadows/python_projects/measleseasles/venv/lib/python3.13/site-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [2]:
defaultConfigURL = 'https://docs.google.com/spreadsheets/d/1zn0e2-hi-p9hcOWiMk03v4TH_zq64BfD9pUgbAf1uRs/edit?usp=sharing'
preprocessorConfig = ep.getGoogleSheetConfig(defaultConfigURL)

In [3]:
runsDf = pd.read_csv('input/beta_run_specs.csv')

In [4]:
mm.preppedCountries = md.prepData(defaultLoc = 'input/processed_measles_model_data.csv',
                               cutoffsCol = 'cutoff_date')
mm.preppedCountries['cutoffs']['NGA']

194/194 included countries found with noted outbreaks.


Timestamp('2023-08-01 00:00:00')

In [5]:
randomSeeds = [1337 * i for i in range(25)]
#geographies = runsDf.ID.unique().tolist() + ['cluster:2']
geographies = runsDf.ID.unique().tolist()
fuzzReplicates = [0, 10]
ensembleModels = ['diverse low n']
ensembleModels = [[], 'diverse', 'diverse low n', 'boosted alpha']



indepVars = {'incoming_air_passengers': 0,
             'mean_precip_mm_per_day': 3,
             'mean_max_temp': 3}

baseModel = {'modelName': 'XGBRegressor','model':XGBRegressor}

In [8]:
genericRun = mm.sklGeneric(countries = ['PAK'],              # Single string - now works!
                           depVar = 'cases_1M',
                           modelArgs = baseModel,
                           randomState = 2371,
                           indepVars = indepVars,
                           fuzzReplicates = 0,
                           ensembleModels = [],
                           preprocessor = preprocessorConfig,
                           metaRow = 1,            
                           prefix = 'test_run')
genericRun.train()
results = genericRun.evaluate()

In [6]:
mm.preppedCountries['curves']['NGA'][['ds'] + list(indepVars.keys())]

,ds,incoming_air_passengers,mean_precip_mm_per_day,mean_max_temp
0,2011-01-01,11117,0.055000,30.603861
1,2011-02-01,10069,1.418750,34.094880
2,2011-03-01,10534,1.046250,36.016330
3,2011-04-01,10125,1.724545,35.258125
4,2011-05-01,10563,3.845455,33.956328
...,...,...,...,...
160,2024-05-01,10563,3.853000,36.453083
161,2024-06-01,10374,8.380000,32.937260
162,2024-07-01,10380,13.479091,30.375189
163,2024-08-01,10698,8.330000,29.378009


In [ ]:
collectedResults = []
for randomSeed in randomSeeds:
    for geography in geographies:
        clear_output()
        for fuzzN in fuzzReplicates:
            for ensembleModel in ensembleModels:
                print(randomSeed,geography,fuzzN,ensembleModel)
                try:
                    genericRun = mm.sklGeneric(geography,
                                               'cases_1M',
                                               modelArgs = baseModel,
                                               randomState = randomSeed,
                                               indepVars = indepVars,
                                               fuzzReplicates = fuzzN,
                                               ensembleModels = ensembleModel)
                    
                    metaFileRef = f'output/beta_methods_eval/{genericRun.hash}_Summary.csv'
                    if not os.path.exists(metaFileRef):
                        try:
                            genericRun.train()
                            results = genericRun.evaluate()
                            results['run time'] = datetime.now().strftime('%y-%m-%d %H:%M:%S')
                            mm.exportTables(genericRun, directory = 'output/beta_methods_eval')
                            collectedResults.append(results)
                        except:
                            print("Run fails at fitting")
                            pass
                    else:
                        results = pd.read_csv(metaFileRef)
                        collectedResults.append(results)
                except:
                    print("Run fails at initialization")





6685 LKA 0 []
6685 LKA 0 diverse


WARNING - (py.warnings._showwarnmsg) - /home/jschlitt/miniforge3/envs/bmgf/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/jschlitt/miniforge3/envs/bmgf/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/jschlitt/miniforge3/envs/bmgf/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/jschlitt/miniforge3/envs/bmgf/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(

WARNING - (py.warnings._showwarnmsg) - /home/jschlitt/miniforge3

6685 LKA 0 diverse low n
6685 LKA 0 boosted alpha


In [8]:
pd.concat(collectedResults).to_csv('BetaMetaData.csv')
! zip -r beta_eval.zip output/beta_methods_eval

In [1]:
#! rm -rf output/beta_methods_eval/*.csv

In [7]:
mergedExperiments.to_csv('output/Merged_Beta_Comparison.csv')

In [14]:
print(mergedExperiments.iloc[200][:50])

Unnamed: 0                                                               LSO
Test MSE                                                            0.383067
Test MAE                                                            0.347695
Test R2                                                            -0.300928
Train MSE                                                           0.000147
Train MAE                                                           0.006183
Train R2                                                            0.999994
Sensitivity                                                              NaN
Specificity                                                              1.0
F1 Score                                                                 NaN
True positives                                                             0
True negatives                                                             9
False positives                                                            0